d ##### Usecase:Get sales data by products
###### Solutions are given for following problem statements
###### Problem statement-1:Get sales data for products per day
###### Problem statement-2:Get sales data for products per month
###### Problem statement-3:Get sales data for products per year
###### Problem statement-4:Get daily sales data from most sold products
###### Problem statement-5:Get third and final quarters of sales data from Top5 revenue generating products for each month
###### Problem statement-6:Get sales data for each product category
###### Problem statement-7:Get monthly sales data from Top5 revenue generating categories
###### Problem statement-8:Get highest product price for each category
###### Problem statement-9:Get the sales data of best-selling and the second best-selling products in every category
###### Problem statement-10:Get TOP5 highest selling product for each year

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum,round,avg,rank, col
from pyspark.sql.window import Window
from pyspark.sql.functions import *
from pyspark.sql.functions import unix_timestamp, lit
spark = SparkSession.builder.master('local').appName('GetSalesDatabyproducts').enableHiveSupport().getOrCreate()

In [3]:
ordersdf = spark.read.csv("/FileStore/tables/retaildbtext/Orders.txt", sep=',',schema ='order_id int,order_date string,order_customer_id int, order_status string')
ordersdf.show(5,False)

+--------+----------+-----------------+---------------+
order_id|order_date|order_customer_id|order_status |
+--------+----------+-----------------+---------------+
1 |2013-07-25|11599 |CLOSED |
2 |2013-07-25|256 |PENDING_PAYMENT|
3 |2013-07-25|12111 |COMPLETE |
4 |2013-07-25|8827 |CLOSED |
5 |2013-07-25|11318 |COMPLETE |
+--------+----------+-----------------+---------------+
only showing top 5 rows

In [4]:
orderItemsdf = spark.read.csv("/FileStore/tables/retaildbtext/Order_Items.txt", sep=',', schema='order_item_id int, order_item_order_id int, order_item_product_id  int, order_item_quantity int,order_item_subtotal float,order_item_product_price float')
orderItemsdf.show(5,False)

+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
order_item_id|order_item_order_id|order_item_product_id|order_item_quantity|order_item_subtotal|order_item_product_price|
+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
1 |1 |957 |1 |299.98 |299.98 |
2 |2 |1073 |1 |199.99 |199.99 |
3 |2 |502 |5 |250.0 |50.0 |
4 |2 |403 |1 |129.99 |129.99 |
5 |4 |897 |2 |49.98 |24.99 |
+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
only showing top 5 rows

In [5]:
customerItemsdf = spark.read.csv("/FileStore/tables/retaildbtext/Customers.txt", sep=';', schema='customer_id int, customer_fname string, customer_lname string, customer_email string,customer_password string,customer_street string,customer_city string, customer_state string, customer_zipcode string')
customerItemsdf.show(5,False)

+-----------+--------------+--------------+--------------+-----------------+-----------------------+-------------+--------------+----------------+
customer_id|customer_fname|customer_lname|customer_email|customer_password|customer_street |customer_city|customer_state|customer_zipcode|
+-----------+--------------+--------------+--------------+-----------------+-----------------------+-------------+--------------+----------------+
1 |Richard |Hernandez |XXXXXXXXX |XXXXXXXXX |6303 Heather Plaza |Brownsville |TX |78521 |
2 |Mary |Barrett |XXXXXXXXX |XXXXXXXXX |9526 Noble Embers Ridge|Littleton |CO |80126 |
3 |Ann |Smith |XXXXXXXXX |XXXXXXXXX |3422 Blue Pioneer Bend |Caguas |PR |00725 |
4 |Mary |Jones |XXXXXXXXX |XXXXXXXXX |8324 Little Common |San Marcos |CA |92069 |
5 |Robert |Hudson |XXXXXXXXX |XXXXXXXXX |10 Crystal River Mall |Caguas |PR |00725 |
+-----------+--------------+--------------+--------------+-----------------+-----------------------+-------------+--------------+----------------+
only showing top 5 rows

In [6]:
productItemsdf = spark.read.csv("/FileStore/tables/retaildbtext/Products.txt", sep=';', schema='product_id int, product_category_id int, product_name string, product_desc string, product_price float, product_image string ')
productItemsdf.show(5,False)

+----------+-------------------+---------------------------------------------+------------+-------------+-------------------------------------------------------------------------------------+
product_id|product_category_id|product_name |product_desc|product_price|product_image |
+----------+-------------------+---------------------------------------------+------------+-------------+-------------------------------------------------------------------------------------+
1 |2 |Quest Q64 10 FT. x 10 FT. Slant Leg Instant U|null |59.98 |http://images.acmesports.sports/Quest+Q64+10+FT.+x+10+FT.+Slant+Leg+Instant+Up+Canopy|
2 |2 |Under Armour Men's Highlight MC Football Clea|null |129.99 |http://images.acmesports.sports/Under+Armour+Men%27s+Highlight+MC+Football+Cleat |
3 |2 |Under Armour Men's Renegade D Mid Football Cl|null |89.99 |http://images.acmesports.sports/Under+Armour+Men%27s+Renegade+D+Mid+Football+Cleat |
4 |2 |Under Armour Men's Renegade D Mid Football Cl|null |89.99 |http://images.acmesports.sports/Under+Armour+Men%27s+Renegade+D+Mid+Football+Cleat |
5 |2 |Riddell Youth Revolution Speed Custom Footbal|null |199.99 |http://images.acmesports.sports/Riddell+Youth+Revolution+Speed+Custom+Football+Helmet|
+----------+-------------------+---------------------------------------------+------------+-------------+-------------------------------------------------------------------------------------+
only showing top 5 rows

In [7]:
categoriesItemsdf = spark.read.csv("/FileStore/tables/retaildbtext/Categories.txt", sep=',', schema='category_id int, category_department_id int, category_name string')
categoriesItemsdf.show(5,False)

+-----------+----------------------+-------------------+
category_id|category_department_id|category_name |
+-----------+----------------------+-------------------+
1 |2 |Football |
2 |2 |Soccer |
3 |2 |Baseball & Softball|
4 |2 |Basketball |
5 |2 |Lacrosse |
+-----------+----------------------+-------------------+
only showing top 5 rows

In [8]:
ordersMap = ordersdf.select('order_id','order_date','order_status')
ordersMap.show(5,False)

+--------+----------+---------------+
order_id|order_date|order_status |
+--------+----------+---------------+
1 |2013-07-25|CLOSED |
2 |2013-07-25|PENDING_PAYMENT|
3 |2013-07-25|COMPLETE |
4 |2013-07-25|CLOSED |
5 |2013-07-25|COMPLETE |
+--------+----------+---------------+
only showing top 5 rows

In [9]:
orderItemsMap=orderItemsdf.select('order_item_order_id','order_item_product_id','order_item_subtotal','order_item_quantity')
orderItemsMap.show(5,False)


+-------------------+---------------------+-------------------+-------------------+
order_item_order_id|order_item_product_id|order_item_subtotal|order_item_quantity|
+-------------------+---------------------+-------------------+-------------------+
1 |957 |299.98 |1 |
2 |1073 |199.99 |1 |
2 |502 |250.0 |5 |
2 |403 |129.99 |1 |
4 |897 |49.98 |2 |
+-------------------+---------------------+-------------------+-------------------+
only showing top 5 rows

In [10]:
ordersdfjoin = ordersMap.join(orderItemsMap, ordersMap.order_id == orderItemsMap.order_item_order_id,'inner')
ordersdfjoin.show(5,False)

+--------+----------+---------------+-------------------+---------------------+-------------------+-------------------+
order_id|order_date|order_status |order_item_order_id|order_item_product_id|order_item_subtotal|order_item_quantity|
+--------+----------+---------------+-------------------+---------------------+-------------------+-------------------+
1 |2013-07-25|CLOSED |1 |957 |299.98 |1 |
2 |2013-07-25|PENDING_PAYMENT|2 |1073 |199.99 |1 |
2 |2013-07-25|PENDING_PAYMENT|2 |502 |250.0 |5 |
2 |2013-07-25|PENDING_PAYMENT|2 |403 |129.99 |1 |
4 |2013-07-25|CLOSED |4 |897 |49.98 |2 |
+--------+----------+---------------+-------------------+---------------------+-------------------+-------------------+
only showing top 5 rows

In [11]:
ordersdfSalesmap = ordersdfjoin.select('order_id','order_date','order_item_product_id','order_status','order_item_subtotal','order_item_quantity')
ordersdfSalesmap.show(10,False)

+--------+----------+---------------------+---------------+-------------------+-------------------+
order_id|order_date|order_item_product_id|order_status |order_item_subtotal|order_item_quantity|
+--------+----------+---------------------+---------------+-------------------+-------------------+
1 |2013-07-25|957 |CLOSED |299.98 |1 |
2 |2013-07-25|1073 |PENDING_PAYMENT|199.99 |1 |
2 |2013-07-25|502 |PENDING_PAYMENT|250.0 |5 |
2 |2013-07-25|403 |PENDING_PAYMENT|129.99 |1 |
4 |2013-07-25|897 |CLOSED |49.98 |2 |
4 |2013-07-25|365 |CLOSED |299.95 |5 |
4 |2013-07-25|502 |CLOSED |150.0 |3 |
4 |2013-07-25|1014 |CLOSED |199.92 |4 |
5 |2013-07-25|957 |COMPLETE |299.98 |1 |
5 |2013-07-25|365 |COMPLETE |299.95 |5 |
+--------+----------+---------------------+---------------+-------------------+-------------------+
only showing top 10 rows

In [12]:
productsMap = productItemsdf.select('product_id','product_category_id','product_name','product_price')
productsMap.show(5,False)

+----------+-------------------+---------------------------------------------+-------------+
product_id|product_category_id|product_name |product_price|
+----------+-------------------+---------------------------------------------+-------------+
1 |2 |Quest Q64 10 FT. x 10 FT. Slant Leg Instant U|59.98 |
2 |2 |Under Armour Men's Highlight MC Football Clea|129.99 |
3 |2 |Under Armour Men's Renegade D Mid Football Cl|89.99 |
4 |2 |Under Armour Men's Renegade D Mid Football Cl|89.99 |
5 |2 |Riddell Youth Revolution Speed Custom Footbal|199.99 |
+----------+-------------------+---------------------------------------------+-------------+
only showing top 5 rows

In [13]:
prodordersdfjoin = ordersdfSalesmap.join(productsMap, ordersdfSalesmap.order_item_product_id == productsMap.product_id,'inner')
prodordersdfjoin.show(5,False)

+--------+----------+---------------------+---------------+-------------------+-------------------+----------+-------------------+---------------------------------------------+-------------+
order_id|order_date|order_item_product_id|order_status |order_item_subtotal|order_item_quantity|product_id|product_category_id|product_name |product_price|
+--------+----------+---------------------+---------------+-------------------+-------------------+----------+-------------------+---------------------------------------------+-------------+
1 |2013-07-25|957 |CLOSED |299.98 |1 |957 |43 |Diamondback Women's Serene Classic Comfort Bi|299.98 |
2 |2013-07-25|1073 |PENDING_PAYMENT|199.99 |1 |1073 |48 |Pelican Sunstream 100 Kayak |199.99 |
2 |2013-07-25|502 |PENDING_PAYMENT|250.0 |5 |502 |24 |Nike Men's Dri-FIT Victory Golf Polo |50.0 |
2 |2013-07-25|403 |PENDING_PAYMENT|129.99 |1 |403 |18 |Nike Men's CJ Elite 2 TD Football Cleat |129.99 |
4 |2013-07-25|897 |CLOSED |49.98 |2 |897 |40 |Team Golf New England Patriots Putter Grip |24.99 |
+--------+----------+---------------------+---------------+-------------------+-------------------+----------+-------------------+---------------------------------------------+-------------+
only showing top 5 rows

In [14]:
prodordersdfsalesmap =prodordersdfjoin.select('order_id','order_date','order_status','order_item_subtotal','order_item_quantity','product_id','product_category_id','product_name','product_price')
prodordersdfsalesmap.show(5,False)


+--------+----------+---------------+-------------------+-------------------+----------+-------------------+---------------------------------------------+-------------+
order_id|order_date|order_status |order_item_subtotal|order_item_quantity|product_id|product_category_id|product_name |product_price|
+--------+----------+---------------+-------------------+-------------------+----------+-------------------+---------------------------------------------+-------------+
1 |2013-07-25|CLOSED |299.98 |1 |957 |43 |Diamondback Women's Serene Classic Comfort Bi|299.98 |
2 |2013-07-25|PENDING_PAYMENT|199.99 |1 |1073 |48 |Pelican Sunstream 100 Kayak |199.99 |
2 |2013-07-25|PENDING_PAYMENT|250.0 |5 |502 |24 |Nike Men's Dri-FIT Victory Golf Polo |50.0 |
2 |2013-07-25|PENDING_PAYMENT|129.99 |1 |403 |18 |Nike Men's CJ Elite 2 TD Football Cleat |129.99 |
4 |2013-07-25|CLOSED |49.98 |2 |897 |40 |Team Golf New England Patriots Putter Grip |24.99 |
+--------+----------+---------------+-------------------+-------------------+----------+-------------------+---------------------------------------------+-------------+
only showing top 5 rows

In [15]:
prodordersdfsalesmap.printSchema()

root
-- order_id: integer (nullable = true)
-- order_date: string (nullable = true)
-- order_status: string (nullable = true)
-- order_item_subtotal: float (nullable = true)
-- order_item_quantity: integer (nullable = true)
-- product_id: integer (nullable = true)
-- product_category_id: integer (nullable = true)
-- product_name: string (nullable = true)
-- product_price: float (nullable = true)

In [16]:
prodcatdfsalesjoin = prodordersdfsalesmap.join(categoriesItemsdf, prodordersdfsalesmap.product_category_id == categoriesItemsdf.category_id,'inner')
prodcatdfsalesjoin.show(5,False)

+--------+----------+---------------+-------------------+-------------------+----------+-------------------+---------------------------------------------+-------------+-----------+----------------------+----------------+
order_id|order_date|order_status |order_item_subtotal|order_item_quantity|product_id|product_category_id|product_name |product_price|category_id|category_department_id|category_name |
+--------+----------+---------------+-------------------+-------------------+----------+-------------------+---------------------------------------------+-------------+-----------+----------------------+----------------+
1 |2013-07-25|CLOSED |299.98 |1 |957 |43 |Diamondback Women's Serene Classic Comfort Bi|299.98 |43 |7 |Camping & Hiking|
2 |2013-07-25|PENDING_PAYMENT|199.99 |1 |1073 |48 |Pelican Sunstream 100 Kayak |199.99 |48 |7 |Water Sports |
2 |2013-07-25|PENDING_PAYMENT|250.0 |5 |502 |24 |Nike Men's Dri-FIT Victory Golf Polo |50.0 |24 |5 |Women's Apparel |
2 |2013-07-25|PENDING_PAYMENT|129.99 |1 |403 |18 |Nike Men's CJ Elite 2 TD Football Cleat |129.99 |18 |4 |Men's Footwear |
4 |2013-07-25|CLOSED |49.98 |2 |897 |40 |Team Golf New England Patriots Putter Grip |24.99 |40 |6 |Accessories |
+--------+----------+---------------+-------------------+-------------------+----------+-------------------+---------------------------------------------+-------------+-----------+----------------------+----------------+
only showing top 5 rows

In [17]:
prodordersdfsalesmap=prodcatdfsalesjoin.select('order_id','order_date','order_status','order_item_subtotal','order_item_quantity','product_id','product_category_id','product_name',\
                                               'product_price','category_name')
prodordersdfsalesmap.show(5,False)

+--------+----------+---------------+-------------------+-------------------+----------+-------------------+---------------------------------------------+-------------+----------------+
order_id|order_date|order_status |order_item_subtotal|order_item_quantity|product_id|product_category_id|product_name |product_price|category_name |
+--------+----------+---------------+-------------------+-------------------+----------+-------------------+---------------------------------------------+-------------+----------------+
1 |2013-07-25|CLOSED |299.98 |1 |957 |43 |Diamondback Women's Serene Classic Comfort Bi|299.98 |Camping & Hiking|
2 |2013-07-25|PENDING_PAYMENT|199.99 |1 |1073 |48 |Pelican Sunstream 100 Kayak |199.99 |Water Sports |
2 |2013-07-25|PENDING_PAYMENT|250.0 |5 |502 |24 |Nike Men's Dri-FIT Victory Golf Polo |50.0 |Women's Apparel |
2 |2013-07-25|PENDING_PAYMENT|129.99 |1 |403 |18 |Nike Men's CJ Elite 2 TD Football Cleat |129.99 |Men's Footwear |
4 |2013-07-25|CLOSED |49.98 |2 |897 |40 |Team Golf New England Patriots Putter Grip |24.99 |Accessories |
+--------+----------+---------------+-------------------+-------------------+----------+-------------------+---------------------------------------------+-------------+----------------+
only showing top 5 rows

###### Problem statement-1:Get sales data for products per day

In [19]:
productsSalesDataPerDay = prodordersdfsalesmap.groupBy('order_date','product_name').agg(round(avg('order_item_subtotal'),2).alias("avgsales"), 
                                                                min('order_item_subtotal').alias("min_sales"),
                                                                max('order_item_subtotal').alias("max_sales"),
                                                                round(sum('order_item_subtotal'),2).alias("Totalsales"),
                                                                sum('order_item_quantity').alias("Totalquantity"),
                                                                count('order_item_subtotal').alias("Numberofsales")).sort('order_date','product_name')
productsSalesDataPerDay.coalesce(1).write.csv("/FileStore/tables/productsSalesDataPerDay", compression="none", header ='true')
productsSalesDataPerDay.show(20,False)

+----------+---------------------------------------------+--------+---------+---------+----------+-------------+-------------+
order_date|product_name |avgsales|min_sales|max_sales|Totalsales|Totalquantity|Numberofsales|
+----------+---------------------------------------------+--------+---------+---------+----------+-------------+-------------+
2013-07-25|Bag Boy Beverage Holder |99.96 |99.96 |99.96 |99.96 |4 |1 |
2013-07-25|Bowflex SelectTech 1090 Dumbbells |599.99 |599.99 |599.99 |599.99 |1 |1 |
2013-07-25|Bridgestone e6 Straight Distance NFL Carolina|63.98 |63.98 |63.98 |63.98 |2 |1 |
2013-07-25|Bridgestone e6 Straight Distance NFL San Dieg|95.97 |95.97 |95.97 |95.97 |3 |1 |
2013-07-25|Bridgestone e6 Straight Distance NFL Tennesse|127.96 |127.96 |127.96 |127.96 |4 |1 |
2013-07-25|Cleveland Golf Women's 588 RTX CB Satin Chrom|119.99 |119.99 |119.99 |239.98 |2 |2 |
2013-07-25|Clicgear 8.0 Shoe Brush |19.98 |19.98 |19.98 |19.98 |2 |1 |
2013-07-25|Diamondback Women's Serene Classic Comfort Bi|299.98 |299.98 |299.98 |9599.36 |32 |32 |
2013-07-25|Elevation Training Mask 2.0 |319.96 |319.96 |319.96 |319.96 |4 |1 |
2013-07-25|Field & Stream Sportsman 16 Gun Fire Safe |399.98 |399.98 |399.98 |10799.46 |27 |27 |
2013-07-25|Glove It Imperial Golf Towel |79.95 |79.95 |79.95 |79.95 |5 |1 |
2013-07-25|Glove It Urban Brick Golf Towel |79.95 |79.95 |79.95 |79.95 |5 |1 |
2013-07-25|Glove It Women's Mod Oval 3-Zip Carry All Gol|21.99 |21.99 |21.99 |21.99 |1 |1 |
2013-07-25|Glove It Women's Mod Oval Golf Glove |79.96 |79.96 |79.96 |79.96 |4 |1 |
2013-07-25|GolfBuddy VT3 GPS Watch |199.99 |199.99 |199.99 |199.99 |1 |1 |
2013-07-25|LIJA Women's Button Golf Dress |108.0 |108.0 |108.0 |216.0 |2 |2 |
2013-07-25|MDGolf Pittsburgh Penguins Putter |239.97 |239.97 |239.97 |239.97 |3 |1 |
2013-07-25|Merrell Men's All Out Flash Trail Running Sho|109.99 |109.99 |109.99 |219.98 |2 |2 |
2013-07-25|Nike Men's CJ Elite 2 TD Football Cleat |129.99 |129.99 |129.99 |5589.57 |43 |43 |
2013-07-25|Nike Men's Dri-FIT Victory Golf Polo |130.77 |50.0 |250.0 |5100.0 |102 |39 |
+----------+---------------------------------------------+--------+---------+---------+----------+-------------+-------------+
only showing top 20 rows

###### Problem statement-2: Get sales data for products per month

In [21]:
productsSalesDataPerMonth = prodordersdfsalesmap.groupBy(substring('order_date',0,7).alias('ordermonth'),'product_name').agg(round(avg('order_item_subtotal'),2).alias("avgsales"), 
                                                                min('order_item_subtotal').alias("min_sales"),
                                                                max('order_item_subtotal').alias("max_sales"),
                                                                round(sum('order_item_subtotal'),2).alias("Totalsales"),
                                                                sum('order_item_quantity').alias("Totalquantity"),
                                                                count('order_item_subtotal').alias("Numberofsales")).sort('ordermonth','product_name')
productsSalesDataPerMonth.coalesce(1).write.csv("/FileStore/tables/productsSalesDataPerMonth", compression="none", header ='true')
productsSalesDataPerMonth.show(20,False)

+----------+---------------------------------------------+--------+---------+---------+----------+-------------+-------------+
ordermonth|product_name |avgsales|min_sales|max_sales|Totalsales|Totalquantity|Numberofsales|
+----------+---------------------------------------------+--------+---------+---------+----------+-------------+-------------+
2013-07 |Bag Boy Beverage Holder |78.54 |24.99 |124.95 |549.78 |22 |7 |
2013-07 |Bowflex SelectTech 1090 Dumbbells |599.99 |599.99 |599.99 |599.99 |1 |1 |
2013-07 |Bridgestone e6 Straight Distance NFL Carolina|81.21 |31.99 |127.96 |1055.67 |33 |13 |
2013-07 |Bridgestone e6 Straight Distance NFL San Dieg|71.98 |31.99 |159.95 |575.82 |18 |8 |
2013-07 |Bridgestone e6 Straight Distance NFL Tennesse|79.97 |31.99 |127.96 |319.9 |10 |4 |
2013-07 |Brooks Women's Ghost 6 Running Shoe |179.98 |179.98 |179.98 |179.98 |2 |1 |
2013-07 |Cleveland Golf Collegiate My Custom Wedge 588|209.99 |209.99 |209.99 |209.99 |1 |1 |
2013-07 |Cleveland Golf Women's 588 RTX CB Satin Chrom|119.99 |119.99 |119.99 |359.97 |3 |3 |
2013-07 |Clicgear 8.0 Shoe Brush |25.97 |9.99 |39.96 |129.87 |13 |5 |
2013-07 |Clicgear Rovic Cooler Bag |159.96 |79.98 |199.95 |639.84 |16 |4 |
2013-07 |Columbia Men's PFG Anchor Tough T-Shirt |102.0 |30.0 |150.0 |510.0 |17 |5 |
2013-07 |Diamondback Boys' Insight 24 Performance Hybr|299.99 |299.99 |299.99 |299.99 |1 |1 |
2013-07 |Diamondback Women's Serene Classic Comfort Bi|299.98 |299.98 |299.98 |95993.6 |320 |320 |
2013-07 |ENO Atlas Hammock Straps |79.97 |29.99 |149.95 |719.76 |24 |9 |
2013-07 |Elevation Training Mask 2.0 |319.96 |319.96 |319.96 |319.96 |4 |1 |
2013-07 |Field & Stream Sportsman 16 Gun Fire Safe |399.98 |399.98 |399.98 |155592.22 |389 |389 |
2013-07 |Fitbit The One Wireless Activity & Sleep Trac|399.8 |399.8 |399.8 |399.8 |4 |1 |
2013-07 |Garmin Approach S3 Golf GPS Watch |249.99 |249.99 |249.99 |249.99 |1 |1 |
2013-07 |Glove It Imperial Golf Towel |71.95 |63.96 |79.95 |143.91 |9 |2 |
2013-07 |Glove It Urban Brick Golf Towel |59.39 |15.99 |79.95 |415.74 |26 |7 |
+----------+---------------------------------------------+--------+---------+---------+----------+-------------+-------------+
only showing top 20 rows

###### Problem statement-3: Get sales data  for products per year

In [23]:
productsSalesDataPerYear = prodordersdfsalesmap.groupBy(substring('order_date',0,4).alias('orderyear'),'product_name').agg(round(avg('order_item_subtotal'),2).alias("avgsales"), 
                                                                min('order_item_subtotal').alias("min_sales"),
                                                                max('order_item_subtotal').alias("max_sales"),
                                                                round(sum('order_item_subtotal'),2).alias("Totalsales"),
                                                                sum('order_item_quantity').alias("Totalquantity"),
                                                                count('order_item_subtotal').alias("Numberofsales")).sort('orderyear','product_name')
productsSalesDataPerYear.coalesce(1).write.csv("/FileStore/tables/productsSalesDataPerYear", compression="none", header ='true')
productsSalesDataPerYear.show(20,False)

+---------+---------------------------------------------+--------+---------+---------+----------+-------------+-------------+
orderyear|product_name |avgsales|min_sales|max_sales|Totalsales|Totalquantity|Numberofsales|
+---------+---------------------------------------------+--------+---------+---------+----------+-------------+-------------+
2013 |Bag Boy Beverage Holder |80.92 |24.99 |124.95 |10195.92 |408 |126 |
2013 |Bag Boy M330 Push Cart |212.47 |79.99 |399.95 |6799.15 |85 |32 |
2013 |Bowflex SelectTech 1090 Dumbbells |599.99 |599.99 |599.99 |4199.93 |7 |7 |
2013 |Bridgestone e6 Straight Distance NFL Carolina|98.58 |31.99 |159.95 |14491.47 |453 |147 |
2013 |Bridgestone e6 Straight Distance NFL San Dieg|89.15 |31.99 |159.95 |12124.21 |379 |136 |
2013 |Bridgestone e6 Straight Distance NFL Tennesse|102.13 |31.99 |159.95 |11132.52 |348 |109 |
2013 |Brooks Women's Ghost 6 Running Shoe |335.73 |89.99 |449.95 |8729.03 |97 |26 |
2013 |Bushnell Pro X7 Jolt Slope Rangefinder |599.99 |599.99 |599.99 |2399.96 |4 |4 |
2013 |Cleveland Golf Collegiate My Custom Wedge 588|209.99 |209.99 |209.99 |6089.71 |29 |29 |
2013 |Cleveland Golf Women's 588 RTX CB Satin Chrom|119.99 |119.99 |119.99 |3359.72 |28 |28 |
2013 |Clicgear 8.0 Shoe Brush |31.71 |9.99 |49.95 |4185.81 |419 |132 |
2013 |Clicgear Rovic Cooler Bag |130.05 |39.99 |199.95 |14956.26 |374 |115 |
2013 |Columbia Men's PFG Anchor Tough T-Shirt |95.52 |30.0 |150.0 |11940.0 |398 |125 |
2013 |Diamondback Boys' Insight 24 Performance Hybr|299.99 |299.99 |299.99 |4199.86 |14 |14 |
2013 |Diamondback Girls' Clarity 24 Hybrid Bike 201|299.99 |299.99 |299.99 |3599.88 |12 |12 |
2013 |Diamondback Women's Serene Classic Comfort Bi|299.98 |299.98 |299.98 |1832277.91|6108 |6108 |
2013 |ENO Atlas Hammock Straps |88.57 |29.99 |149.95 |13285.57 |443 |150 |
2013 |Elevation Training Mask 2.0 |254.71 |79.99 |399.95 |9678.79 |121 |38 |
2013 |Field & Stream Sportsman 16 Gun Fire Safe |399.98 |399.98 |399.98 |3076246.26|7691 |7691 |
2013 |Fitbit The One Wireless Activity & Sleep Trac|295.31 |99.95 |499.75 |6496.75 |65 |22 |
+---------+---------------------------------------------+--------+---------+---------+----------+-------------+-------------+
only showing top 20 rows

###### Problem statement-4:Get daily sales data from most sold products

In [25]:
soldproductsMap =  prodordersdfsalesmap.groupBy('order_date','product_id','product_name').agg(sum('order_item_quantity').alias("soldproductsperday"))
soldproductsMapsorted = soldproductsMap.sort('order_date','soldproductsperday',ascending=[1,0,0])
soldproductsMapsorted.coalesce(1).write.csv("/FileStore/tables/MostSoldProductseachday", compression="none", header ='true')
soldproductsMapsorted.show(20, False)

+----------+----------+---------------------------------------------+------------------+
order_date|product_id|product_name |soldproductsperday|
+----------+----------+---------------------------------------------+------------------+
2013-07-25|1014 |O'Brien Men's Neoprene Life Vest |128 |
2013-07-25|365 |Perfect Fitness Perfect Rip Deck |126 |
2013-07-25|502 |Nike Men's Dri-FIT Victory Golf Polo |102 |
2013-07-25|191 |Nike Men's Free 5.0+ Running Shoe |85 |
2013-07-25|627 |Under Armour Girls' Toddler Spine Surge Runni|72 |
2013-07-25|403 |Nike Men's CJ Elite 2 TD Football Cleat |43 |
2013-07-25|1073 |Pelican Sunstream 100 Kayak |35 |
2013-07-25|957 |Diamondback Women's Serene Classic Comfort Bi|32 |
2013-07-25|1004 |Field & Stream Sportsman 16 Gun Fire Safe |27 |
2013-07-25|276 |Under Armour Women's Ignite Slide |8 |
2013-07-25|886 |Team Golf San Francisco Giants Putter Grip |7 |
2013-07-25|897 |Team Golf New England Patriots Putter Grip |5 |
2013-07-25|926 |Glove It Imperial Golf Towel |5 |
2013-07-25|924 |Glove It Urban Brick Golf Towel |5 |
2013-07-25|24 |Elevation Training Mask 2.0 |4 |
2013-07-25|906 |Team Golf Tennessee Volunteers Putter Grip |4 |
2013-07-25|821 |Titleist Pro V1 High Numbers Personalized Gol|4 |
2013-07-25|134 |Nike Women's Legend V-Neck T-Shirt |4 |
2013-07-25|810 |Glove It Women's Mod Oval Golf Glove |4 |
2013-07-25|778 |Bag Boy Beverage Holder |4 |
+----------+----------+---------------------------------------------+------------------+
only showing top 20 rows

###### Problem statement-5:Get third and final quarters of 2013 sales data from Top5 revenue generating products for each month

In [27]:
SalesDataPerMonth=prodordersdfsalesmap.where((to_date(prodordersdfsalesmap.order_date) >=lit("2013-10-01")) & 
                                                 (to_date(prodordersdfsalesmap.order_date) <=lit("2013-12-31")))

productsSalesDataPerMonth = SalesDataPerMonth.groupBy(substring('order_date',0,7).alias('ordermonth'),'product_name').agg(round(avg('order_item_subtotal'),2).alias("avgsales"), 
                                                                round(sum('order_item_subtotal'),2).alias("Totalsales"),
                                                                sum('order_item_quantity').alias("Totalquantity"),
                                                                count('order_item_subtotal').alias("Numberofsales")).sort('ordermonth','product_name')

window = Window.partitionBy(productsSalesDataPerMonth['ordermonth']).orderBy(productsSalesDataPerMonth['Totalsales'].desc())
top5productspurchasedeachmonth=productsSalesDataPerMonth.select('ordermonth', 'product_name', 'Totalsales').withColumn('rank', rank().over(window).alias('rank'))
top5rankedproductseachmonth = top5productspurchasedeachmonth.filter(col('rank')<=5).sort('ordermonth','Totalsales',ascending=[1,0])
top5rankedproductseachmonth.coalesce(1).write.csv("/FileStore/tables/top5rankedproductseachmonth", compression="none", header ='true')
top5rankedproductseachmonth.show(20,False)

+----------+---------------------------------------------+----------+----+
ordermonth|product_name |Totalsales|rank|
+----------+---------------------------------------------+----------+----+
2013-10 |Field & Stream Sportsman 16 Gun Fire Safe |524773.77 |1 |
2013-10 |Perfect Fitness Perfect Rip Deck |341883.02 |2 |
2013-10 |Diamondback Women's Serene Classic Comfort Bi|310779.29 |3 |
2013-10 |Nike Men's Free 5.0+ Running Shoe |274672.53 |4 |
2013-10 |Nike Men's Dri-FIT Victory Golf Polo |247050.0 |5 |
2013-11 |Field & Stream Sportsman 16 Gun Fire Safe |631568.44 |1 |
2013-11 |Perfect Fitness Perfect Rip Deck |404632.56 |2 |
2013-11 |Diamondback Women's Serene Classic Comfort Bi|389674.03 |3 |
2013-11 |Nike Men's Free 5.0+ Running Shoe |343365.66 |4 |
2013-11 |Nike Men's Dri-FIT Victory Golf Polo |291500.0 |5 |
2013-12 |Field & Stream Sportsman 16 Gun Fire Safe |618769.08 |1 |
2013-12 |Perfect Fitness Perfect Rip Deck |358920.18 |2 |
2013-12 |Diamondback Women's Serene Classic Comfort Bi|350676.63 |3 |
2013-12 |Nike Men's Free 5.0+ Running Shoe |312068.79 |4 |
2013-12 |Pelican Sunstream 100 Kayak |262786.87 |5 |
+----------+---------------------------------------------+----------+----+

###### Problem statement-6:Get sales data for each product category

In [29]:
productsSalesDataPercategory = prodordersdfsalesmap.groupBy('product_category_id','category_name').agg(round(avg('order_item_subtotal'),2).alias("avgsales"), 
                                                                round(sum('order_item_subtotal'),2).alias("Totalsales"),
                                                                sum('order_item_quantity').alias("Totalquantity"),
                                                                count('order_item_subtotal').alias("Numberofsales")).sort('product_category_id','category_name')
productsSalesDataPercategory.coalesce(1).write.csv("/FileStore/tables/productsSalesDataPercategory", compression="none", header ='true')
productsSalesDataPercategory.show(60,False)

+-------------------+--------------------+--------+----------+-------------+-------------+
product_category_id|category_name |avgsales|Totalsales|Totalquantity|Numberofsales|
+-------------------+--------------------+--------+----------+-------------+-------------+
2 |Soccer |191.86 |26477.05 |295 |138 |
3 |Baseball & Softball |148.82 |94057.15 |1785 |632 |
4 |Basketball |404.47 |27099.33 |67 |67 |
5 |Lacrosse |115.06 |39464.79 |1021 |343 |
6 |Tennis & Racquet |135.93 |44585.09 |991 |328 |
7 |Hockey |78.76 |48360.73 |1711 |614 |
9 |Cardio Equipment |295.9 |3694843.2 |37587 |12487 |
10 |Strength Training |494.55 |54895.53 |111 |111 |
11 |Fitness Accessories |115.22 |35601.44 |856 |309 |
12 |Boxing & MMA |201.43 |85205.41 |1265 |423 |
13 |Electronics |102.36 |115355.25 |3375 |1127 |
16 |As Seen on TV! |302.91 |20597.94 |206 |68 |
17 |Cleats |180.52 |4431942.78|73734 |24551 |
18 |Men's Footwear |129.99 |2891757.66|22246 |22246 |
24 |Women's Apparel |149.65 |3147800.0 |62956 |21035 |
26 |Girls' Apparel |126.32 |151706.2 |3615 |1201 |
29 |Shop By Sport |119.22 |1309522.04|32726 |10984 |
30 |Men's Golf Clubs |166.2 |47035.8 |420 |283 |
31 |Women's Golf Clubs |246.11 |44545.97 |403 |181 |
32 |Golf Apparel |79.3 |34969.75 |1025 |441 |
33 |Golf Shoes |206.1 |107998.0 |1444 |524 |
34 |Golf Bags & Carts |169.99 |10369.39 |61 |61 |
35 |Golf Gloves |109.06 |116695.39 |2961 |1070 |
36 |Golf Balls |52.27 |77098.16 |4384 |1475 |
37 |Electronics |126.01 |255679.39 |6061 |2029 |
38 |Kids' Golf Clubs |257.29 |98797.58 |645 |384 |
40 |Accessories |75.1 |133671.51 |5349 |1780 |
41 |Trade-In |70.56 |68721.78 |2822 |974 |
43 |Camping & Hiking |299.98 |4118425.57|13729 |13729 |
44 |Hunting & Shooting |129.2 |56848.42 |1239 |440 |
45 |Fishing |399.98 |6929653.69|17325 |17325 |
46 |Indoor/Outdoor Games|149.7 |2888993.91|57803 |19298 |
48 |Water Sports |200.38 |3113844.68|15540 |15540 |
+-------------------+--------------------+--------+----------+-------------+-------------+

###### Problem statement-7:Get monthly sales data from Top5 revenue generating categories

In [31]:
prodSalesDatatop5catg = prodordersdfsalesmap.groupBy(substring('order_date',0,7).alias("ordermonth"),'product_category_id','category_name').\
                                                                agg(round(avg('order_item_subtotal'),2).alias("avgsales"), 
                                                                round(sum('order_item_subtotal'),2).alias("Totalsales"),
                                                                sum('order_item_quantity').alias("Totalquantity"),
                                                                count('order_item_subtotal').alias("Numberofsales")).sort('ordermonth','product_category_id','category_name')
window = Window.partitionBy(prodSalesDatatop5catg['ordermonth']).orderBy(prodSalesDatatop5catg['Totalsales'].desc())
top5productscategorieseachmonth=prodSalesDatatop5catg.select('ordermonth','product_category_id','category_name','Totalsales').withColumn('rank', rank().over(window).alias('rank'))
top5rankedproductscategorieseachmonth = top5productscategorieseachmonth.filter(col('rank')<=5).sort('ordermonth','Totalsales',ascending=[1,0]) 
top5rankedproductscategorieseachmonth.coalesce(1).write.csv("/FileStore/tables/top5rankedproductscategorieseachmonth", compression="none", header ='true')
top5rankedproductscategorieseachmonth.show(60,False)

+----------+-------------------+--------------------+----------+----+
ordermonth|product_category_id|category_name |Totalsales|rank|
+----------+-------------------+--------------------+----------+----+
2013-07 |45 |Fishing |155592.22 |1 |
2013-07 |17 |Cleats |103182.88 |2 |
2013-07 |43 |Camping & Hiking |95993.6 |3 |
2013-07 |9 |Cardio Equipment |79132.17 |4 |
2013-07 |46 |Indoor/Outdoor Games|67722.9 |5 |
2013-08 |45 |Fishing |562371.9 |1 |
2013-08 |17 |Cleats |367079.06 |2 |
2013-08 |43 |Camping & Hiking |332677.83 |3 |
2013-08 |9 |Cardio Equipment |295210.71 |4 |
2013-08 |48 |Water Sports |262686.89 |5 |
2013-09 |45 |Fishing |583170.86 |1 |
2013-09 |17 |Cleats |370318.44 |2 |
2013-09 |43 |Camping & Hiking |352476.51 |3 |
2013-09 |9 |Cardio Equipment |342925.98 |4 |
2013-09 |48 |Water Sports |269636.55 |5 |
2013-10 |45 |Fishing |524773.77 |1 |
2013-10 |17 |Cleats |343382.97 |2 |
2013-10 |43 |Camping & Hiking |310779.29 |3 |
2013-10 |9 |Cardio Equipment |276742.53 |4 |
2013-10 |24 |Women's Apparel |247050.0 |5 |
2013-11 |45 |Fishing |631568.44 |1 |
2013-11 |17 |Cleats |404632.56 |2 |
2013-11 |43 |Camping & Hiking |389674.03 |3 |
2013-11 |9 |Cardio Equipment |346395.66 |4 |
2013-11 |24 |Women's Apparel |291500.0 |5 |
2013-12 |45 |Fishing |618769.08 |1 |
2013-12 |17 |Cleats |359820.15 |2 |
2013-12 |43 |Camping & Hiking |350676.63 |3 |
2013-12 |9 |Cardio Equipment |313868.79 |4 |
2013-12 |48 |Water Sports |264186.83 |5 |
2014-01 |45 |Fishing |615969.22 |1 |
2014-01 |17 |Cleats |382136.55 |2 |
2014-01 |43 |Camping & Hiking |341677.23 |3 |
2014-01 |9 |Cardio Equipment |302949.93 |4 |
2014-01 |24 |Women's Apparel |268450.0 |5 |
2014-02 |45 |Fishing |553572.34 |1 |
2014-02 |17 |Cleats |379136.85 |2 |
2014-02 |43 |Camping & Hiking |332977.81 |3 |
2014-02 |9 |Cardio Equipment |282161.91 |4 |
2014-02 |48 |Water Sports |262736.91 |5 |
2014-03 |45 |Fishing |580770.98 |1 |
2014-03 |17 |Cleats |367138.85 |2 |
2014-03 |43 |Camping & Hiking |350076.67 |3 |
2014-03 |9 |Cardio Equipment |313928.85 |4 |
2014-03 |24 |Women's Apparel |270800.0 |5 |
2014-04 |45 |Fishing |571171.46 |1 |
2014-04 |17 |Cleats |358380.39 |2 |
2014-04 |43 |Camping & Hiking |326378.25 |3 |
2014-04 |9 |Cardio Equipment |287381.46 |4 |
2014-04 |24 |Women's Apparel |264600.0 |5 |
2014-05 |45 |Fishing |555172.26 |1 |
2014-05 |17 |Cleats |361079.86 |2 |
2014-05 |43 |Camping & Hiking |326978.21 |3 |
2014-05 |9 |Cardio Equipment |303509.88 |4 |
2014-05 |48 |Water Sports |253237.4 |5 |
2014-06 |45 |Fishing |540772.97 |1 |
2014-06 |17 |Cleats |352921.26 |2 |
2014-06 |43 |Camping & Hiking |318878.75 |3 |
2014-06 |9 |Cardio Equipment |317858.37 |4 |
2014-06 |24 |Women's Apparel |253750.0 |5 |
+----------+-------------------+--------------------+----------+----+
only showing top 60 rows

###### Problem statement-8:Get highest product price for each category

In [33]:
productsSalesDatahighprice = prodordersdfsalesmap.groupBy('product_category_id','category_name').agg(max('product_price').alias("highestproductprice"))                                                                                                          .sort('product_category_id','category_name')
productsSalesDatahighprice.coalesce(1).write.csv("/FileStore/tables/productsSalesDatahighprice", compression="none", header ='true')
productsSalesDatahighprice.show(60,False)

+-------------------+--------------------+-------------------+
product_category_id|category_name |highestproductprice|
+-------------------+--------------------+-------------------+
2 |Soccer |124.99 |
3 |Baseball & Softball |159.99 |
4 |Basketball |999.99 |
5 |Lacrosse |99.99 |
6 |Tennis & Racquet |44.99 |
7 |Hockey |329.99 |
9 |Cardio Equipment |99.99 |
10 |Strength Training |1999.99 |
11 |Fitness Accessories |599.99 |
12 |Boxing & MMA |94.99 |
13 |Electronics |44.99 |
16 |As Seen on TV! |99.99 |
17 |Cleats |299.99 |
18 |Men's Footwear |129.99 |
24 |Women's Apparel |50.0 |
26 |Girls' Apparel |70.0 |
29 |Shop By Sport |199.99 |
30 |Men's Golf Clubs |134.99 |
31 |Women's Golf Clubs |209.99 |
32 |Golf Apparel |129.99 |
33 |Golf Shoes |108.0 |
34 |Golf Bags & Carts |169.99 |
35 |Golf Gloves |299.99 |
36 |Golf Balls |19.99 |
37 |Electronics |51.99 |
38 |Kids' Golf Clubs |599.99 |
40 |Accessories |24.99 |
41 |Trade-In |249.99 |
43 |Camping & Hiking |299.98 |
44 |Hunting & Shooting |149.99 |
45 |Fishing |399.98 |
46 |Indoor/Outdoor Games|49.98 |
48 |Water Sports |349.99 |
+-------------------+--------------------+-------------------+

###### Problem statement-9: Get the sales data of best-selling and the second best-selling products in every category

In [35]:
productSalesDatabstsellcategories = prodordersdfsalesmap.groupBy('product_category_id','category_name','product_name').\
                                                                agg(round(avg('order_item_subtotal'),2).alias("avgsales"), 
                                                                round(sum('order_item_subtotal'),2).alias("Totalsales"),
                                                                sum('order_item_quantity').alias("Totalquantity"),
                                                                count('order_item_subtotal').alias("Numberofsales")).sort('product_category_id','category_name','product_name')
window = Window.partitionBy(productSalesDatabstsellcategories['product_category_id']).orderBy(productSalesDatabstsellcategories['Totalsales'].desc())
top2productscategorieseachmonth=productSalesDatabstsellcategories.select('product_category_id','category_name','product_name','Totalsales').withColumn('rank', rank().over(window).alias('rank'))
top2rankedproductscategorieseachmonth = top2productscategorieseachmonth.filter(col('rank')<=2).sort('product_category_id','Totalsales',ascending=[1,0]) 
top2rankedproductscategorieseachmonth.coalesce(1).write.csv("/FileStore/tables/top2rankedproductscategorieseachmonth", compression="none", header ='true')
top2rankedproductscategorieseachmonth.show(60,False)

+-------------------+--------------------+---------------------------------------------+----------+----+
product_category_id|category_name |product_name |Totalsales|rank|
+-------------------+--------------------+---------------------------------------------+----------+----+
2 |Soccer |Elevation Training Mask 2.0 |18477.69 |1 |
2 |Soccer |Nike Men's Fingertrap Max Training Shoe |7999.36 |2 |
3 |Baseball & Softball |adidas Men's F10 Messi TRX FG Soccer Cleat |56330.61 |1 |
3 |Baseball & Softball |adidas Kids' F5 Messi FG Soccer Cleat |27327.19 |2 |
4 |Basketball |SOLE E25 Elliptical |9999.9 |1 |
4 |Basketball |Diamondback Boys' Insight 24 Performance Hybr|8699.71 |2 |
5 |Lacrosse |Under Armour Men's Tech II T-Shirt |20866.65 |1 |
5 |Lacrosse |Nike Kids' Grade School KD VI Basketball Shoe|18598.14 |2 |
6 |Tennis & Racquet |Nike Men's Comfort 2 Slide |44585.09 |1 |
7 |Hockey |Nike Women's Legend V-Neck T-Shirt |20025.0 |1 |
7 |Hockey |Nike Dri-FIT Crew Sock 6 Pack |19426.0 |2 |
9 |Cardio Equipment |Nike Men's Free 5.0+ Running Shoe |3667633.2 |1 |
9 |Cardio Equipment |Nike Women's Tempo Shorts |27210.0 |2 |
10 |Strength Training |SOLE E35 Elliptical |29999.85 |1 |
10 |Strength Training |GoPro HERO3+ Black Edition Camera |12799.68 |2 |
11 |Fitness Accessories |Under Armour Hustle Storm Medium Duffle Bag |29601.54 |1 |
11 |Fitness Accessories |Bowflex SelectTech 1090 Dumbbells |5999.9 |2 |
12 |Boxing & MMA |Under Armour Women's Micro G Skulpt Running S|46559.59 |1 |
12 |Boxing & MMA |Nike Women's Free 5.0 TR FIT PRT 4 Training S|19567.94 |2 |
13 |Electronics |Under Armour Men's Compression EV SL Slide |36576.87 |1 |
13 |Electronics |Under Armour Women's Ignite Slide |29398.81 |2 |
16 |As Seen on TV! |Nike Men's Free TR 5.0 TB Training Shoe |20597.94 |1 |
17 |Cleats |Perfect Fitness Perfect Rip Deck |4421143.14|1 |
17 |Cleats |Total Gym 1400 |10799.64 |2 |
18 |Men's Footwear |Nike Men's CJ Elite 2 TD Football Cleat |2891757.66|1 |
24 |Women's Apparel |Nike Men's Dri-FIT Victory Golf Polo |3147800.0 |1 |
26 |Girls' Apparel |adidas Youth Germany Black/Red Away Match Soc|67830.0 |1 |
26 |Girls' Apparel |TYR Boys' Team Digi Jammer |35191.2 |2 |
29 |Shop By Sport |Under Armour Girls' Toddler Spine Surge Runni|1269082.67|1 |
29 |Shop By Sport |Columbia Men's PFG Anchor Tough T-Shirt |27840.0 |2 |
30 |Men's Golf Clubs |Merrell Women's Grassbow Sport Hiking Shoe |20697.93 |1 |
30 |Men's Golf Clubs |Merrell Women's Siren Mid Waterproof Hiking B|9449.3 |2 |
31 |Women's Golf Clubs |TaylorMade White Smoke IN-12 Putter |19298.07 |1 |
31 |Women's Golf Clubs |Cleveland Golf Collegiate My Custom Wedge 588|13649.35 |2 |
32 |Golf Apparel |Top Flite Women's 2014 XL Hybrid |17731.13 |1 |
32 |Golf Apparel |TaylorMade Women's RBZ SL Rescue |8839.32 |2 |
33 |Golf Shoes |LIJA Women's Eyelet Sleeveless Golf Polo |61490.0 |1 |
33 |Golf Shoes |LIJA Women's Mid-Length Panel Golf Shorts |22700.0 |2 |
34 |Golf Bags & Carts |Ogio Race Golf Shoes |10369.39 |1 |
35 |Golf Gloves |Clicgear Rovic Cooler Bag |34671.33 |1 |
35 |Golf Gloves |Bag Boy Beverage Holder |21116.55 |2 |
36 |Golf Balls |Glove It Women's Imperial Golf Glove |18810.59 |1 |
36 |Golf Balls |Glove It Women's Mod Oval Golf Glove |16031.98 |2 |
37 |Electronics |Titleist Pro V1x High Numbers Personalized Go|47206.92 |1 |
37 |Electronics |Titleist Pro V1 High Numbers Personalized Gol|44243.49 |2 |
38 |Kids' Golf Clubs |Fitbit The One Wireless Activity & Sleep Trac|18690.65 |1 |
38 |Kids' Golf Clubs |Polar FT4 Heart Rate Monitor |17638.04 |2 |
40 |Accessories |Team Golf St. Louis Cardinals Putter Grip |23940.42 |1 |
40 |Accessories |Team Golf Tennessee Volunteers Putter Grip |22865.85 |2 |
41 |Trade-In |Glove It Women's Mod Oval 3-Zip Carry All Gol|20450.7 |1 |
41 |Trade-In |Garmin Approach S3 Golf GPS Watch |19249.23 |2 |
43 |Camping & Hiking |Diamondback Women's Serene Classic Comfort Bi|4118425.57|1 |
44 |Hunting & Shooting |ENO Atlas Hammock Straps |29930.02 |1 |
44 |Hunting & Shooting |The North Face

###### Problem statement-10:Get TOP5 highest selling product for each year

In [37]:
productsSalesDatatop5peryear = prodordersdfsalesmap.groupBy(substring('order_date',0,4).alias("orderyear"),'product_name').\
                                                                agg(round(avg('order_item_subtotal'),2).alias("avgsales"), 
                                                                round(sum('order_item_subtotal'),2).alias("Totalsales"),
                                                                sum('order_item_quantity').alias("Totalquantity"),
                                                                count('order_item_subtotal').alias("Numberofsales")).sort('orderyear','product_name','Totalsales')
window = Window.partitionBy(productsSalesDatatop5peryear['orderyear']).orderBy(productsSalesDatatop5peryear['Totalsales'].desc())
top5productscategorieseachmonth=productsSalesDatatop5peryear.select('orderyear','product_name','Totalsales').withColumn('rank', rank().over(window).alias('rank'))
top5rankedproductscategorieseachmonth = top5productscategorieseachmonth.filter(col('rank')<=5).sort('orderyear','Totalsales',ascending=[1,0]) 
top5rankedproductscategorieseachmonth.coalesce(1).write.csv("/FileStore/tables/top5rankedproductscategorieseachmonth", compression="none", header ='true')
top5rankedproductscategorieseachmonth.show(60,False)

+---------+---------------------------------------------+----------+----+
orderyear|product_name |Totalsales|rank|
+---------+---------------------------------------------+----------+----+
2013 |Field & Stream Sportsman 16 Gun Fire Safe |3076246.26|1 |
2013 |Perfect Fitness Perfect Rip Deck |1942416.26|2 |
2013 |Diamondback Women's Serene Classic Comfort Bi|1832277.91|3 |
2013 |Nike Men's Free 5.0+ Running Shoe |1641435.84|4 |
2013 |Nike Men's Dri-FIT Victory Golf Polo |1393000.0 |5 |
2014 |Field & Stream Sportsman 16 Gun Fire Safe |3853407.43|1 |
2014 |Perfect Fitness Perfect Rip Deck |2478726.88|2 |
2014 |Diamondback Women's Serene Classic Comfort Bi|2286147.66|3 |
2014 |Nike Men's Free 5.0+ Running Shoe |2026197.36|4 |
2014 |Nike Men's Dri-FIT Victory Golf Polo |1754800.0 |5 |
+---------+---------------------------------------------+----------+----+